In [1]:
import h5py
import numpy as np
from random import shuffle
from math import ceil
import matplotlib.pyplot as plt

In [2]:
hdf5_path = './dataset.hdf5'
batch_size = 50
nb_class = 2

In [3]:
def load_data(hdf5_path):
    with h5py.File(hdf5_path, 'r') as hdf5_file:
        train_images = np.array(hdf5_file['training_set_img'])
        train_labels = np.array(hdf5_file['training_set_labels'])
        val_images = np.array(hdf5_file['valid_set_img'])
        val_labels = np.array(hdf5_file['valid_set_labels'])
        test_images = np.array(hdf5_file['test_set_img'])
        test_labels = np.array(hdf5_file['test_set_labels'])

    return (train_images, train_labels), (val_images, val_labels), (test_images, test_labels)

In [4]:
(train_images, train_labels), (val_images, val_labels), (test_images, test_labels) = load_data(hdf5_path)

In [5]:
# Converts categorical labels into integers.
from sklearn.preprocessing import LabelEncoder

train_labels = np.array([label.decode('utf-8') for label in train_labels])
val_labels = np.array([label.decode('utf-8') for label in val_labels])
test_labels = np.array([label.decode('utf-8') for label in test_labels])

encoder = LabelEncoder()
train_labels = encoder.fit_transform(train_labels)
val_labels = encoder.transform(val_labels)
test_labels = encoder.transform(test_labels)

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input

model = Sequential([
    Input(shape=(300, 300, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(len(encoder.classes_), activation='softmax')
])

In [7]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 298, 298, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 149, 149, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 147, 147, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 73, 73, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 71, 71, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 35, 35, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 156800)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │    80,282,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 26)             │        13,338 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 80,388,698 (306.66 MB)

 Trainable params: 80,388,698 (306.66 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
model.fit(train_images, train_labels, epochs=10, validation_data=(val_images, val_labels))

# Evaluate the model
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)
print(f'Test accuracy: {test_acc}')

Epoch 1/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 91s 924ms/step - accuracy: 0.6595 - loss: 438.8541 - val_accuracy: 0.9788 - val_loss: 0.0705
Epoch 2/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 87s 891ms/step - accuracy: 0.9981 - loss: 0.0107 - val_accuracy: 0.9942 - val_loss: 0.0349
Epoch 3/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 87s 889ms/step - accuracy: 1.0000 - loss: 1.3353e-05 - val_accuracy: 0.9942 - val_loss: 0.0296
Epoch 4/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 87s 893ms/step - accuracy: 1.0000 - loss: 1.2326e-06 - val_accuracy: 0.9942 - val_loss: 0.0289
Epoch 5/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 88s 895ms/step - accuracy: 1.0000 - loss: 9.3228e-07 - val_accuracy: 0.9942 - val_loss: 0.0287
Epoch 6/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 98s 997ms/step - accuracy: 1.0000 - loss: 6.4349e-07 - val_accuracy: 0.9942 - val_loss: 0.0287
Epoch 7/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 89s 903ms/step - accuracy: 1.0000 - loss: 4.4682e-07 - val_accuracy: 0.9942 - val_loss: 0.0287
Epoch 8/10
98/98 ━━━━━━━━━━━━━━━━━━━━ 89s 906ms/step - accuracy: 1.0000 - l

In [10]:
# Save the model
model.save('hand_signal_model.keras')